#   _Key Derivation Function_ [LBK, Kollbi]
###   Podstawowe informacje
1.  Funkcje KDF to algorytmy kryptograficzne zwracające sekret lub wiele sekretów z tajnej wartości.    
        Np. Stworzenie klucza prywatnego (jeden sekret) z hasła użytkownika (tajna wartość).
1.  Metody te powstały ponieważ hasła zazwyczaj są proste i do przewidzenia. Pojawia się potrzeba stworzenia mocnego klucza.  
Jednak nikt nie będzie pamiętał 64 znakowego hasła. Dlatego z pomocą przychodzi `key stretching`. Czyli proces w którym tworzony jest bezpieczniejszy i (teoretycznie) odporniejszy na ataki brute-force nowy klucz.  
Dostając proste hasło zostają wykonane operacje, które dosłownie `rozciągają` podany ciąg znaków. Do key streching'u można zaliczyć proste funkcje hashujące jak SHA, MD5 lub szyfry blokowe.
```
                                               _             _  ______  _____                                   _       
     _ __   __ _ ___ _____      _____  _ __ __| |           | |/ /  _ \|  ___|           ___  ___  ___ _ __ ___| |_ ___  
    | '_ \ / _` / __/ __\ \ /\ / / _ \| '__/ _` |   =====\  | ' /| | | | |_     ====\   / __|/ _ \/ __| '__/ _ \ __/ __|
    | |_) | (_| \__ \__ \\ V  V / (_) | | | (_| |   =====/  | . \| |_| |  _|    ====/   \__ \  __/ (__| | |  __/ |_\__ \
    | .__/ \__,_|___/___/ \_/\_/ \___/|_|  \__,_|           |_|\_\____/|_|              |___/\___|\___|_|  \___|\__|___/
    |_|                                                     
```

### Funkcje KDF to nie tylko funkcje hashujące! Jednak mają wspólną część.
1. Tak samo jak funkcje hashujące szyfrują dane deterministycznie. (To samo wejście - to samo wyjście)
1. Na wyjściu jest zawsze ciąg o takiej samej długości. Bez znaczenia jak duże były dane wejściowe.
1. Działa tylko w jedną stronę. Nie odzyska wejścia danych z wyjścia.


### Lepsze zrozumienie od podstaw
Najprostszą funkcję KDF można zrobić wykorzystując np. algorytm SHA256 do hashowania hasła. Jednak jest to niebezpieczne. Proste hashe są podatne na ataki słownikowe (np. tęczowe tablice (Rainbow table))

function_sha256(`password`) --> `key`

Pierwszym krokiem w stronę KDF (i zabezpieczeniem) jest dodanie `soli` (ang. `salt`).  
`Salt` jest losową wartością przechowywaną razem z uzyskanym kluczem. Używana, żeby pozyskać ten sam klucz z podanego hasła.

function_sha256_with_salt(`salt`, `password`, `sha256`) --> `key`

Funkcję wyżej można opisać jako `HKDF` (`HMAC-based key derivation function`). Ciągle jest mniej bezpieczna niż aktualniejsze KDF. Zaleca się korzystanie z PBKDF2, Bcrypt, Scrypt (ten projekt) oraz Argon2.


In [6]:
from hmac import HMAC
from hashlib import sha256

salt = b'Crypto'
password = b'IsFun'

key = HMAC(salt, password, digestmod=sha256)

print(key.digest().hex())

a773d6dd2e141ddc22a578716a5da01e695286e03abf3b09abb3c48c296e1a70
